Оценка RAG

In [ ]:
from typing import List
import os
import asyncio
import json
import numpy as np
from react_agent.tools import search 
from react_agent import graph
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from ragas import EvaluationDataset, evaluate
from ragas.llms import LangchainLLMWrapper
from ragas.metrics import LLMContextRecall, Faithfulness, ResponseRelevancy



def retrieve(query: str, top_k: int = 10) -> List[str]:
    """
    Вызов Qdrant-инструмента search() для получения до top_k фрагментов контекста.
    """
    raw = search(query)
    # search() возвращает строки, разделённые сериями ===== и заголовками источников
    # Разбиваем по заголовку каждого результата
    parts = [part.strip() for part in raw.split("==========Актуально в период:") if part.strip()]
    # Берём первые top_k результатов
    return parts[:top_k]


In [14]:
print(graph.input_schema)


<class 'langchain_core.utils.pydantic.ReAct AgentInput'>


Создание датасета

In [19]:
import json

dataset = []
questions = open('../data/testing_data/test_cases.txt', encoding='utf-8').read().splitlines()

for q in questions:
    contexts = retrieve(q)  # список строк

    # 1) Собираем единый prompt
    prompt = (
        "Вопрос пользователя:\n"
        f"{q}"
    )

    # 2) Вызываем агента
    response_obj = await graph.ainvoke({
        "input": prompt,
        "chat_history": []
    })

    # 3) Достаём текст
    response_text = getattr(response_obj, "content", str(response_obj))

    dataset.append({
        "user_input": q,
        "retrieved_contexts": contexts,
        "response": response_text
    })

    print(q, "→", response_text)

# Оставляем первые 10 и сохраняем
dataset = dataset[:10]
with open('dataset.json', 'w', encoding='utf-8') as f:
    json.dump(dataset, f, ensure_ascii=False, indent=4)


Здравствуйте, а подскажите флюорография и справка с анализами от дерматолога сколько действительны, т.е, за сколько до заселения должны быть сделаны? → {'messages': [AIMessage(content='Здравствуйте! Я AI-ассистент приёмной комиссии МАИ. Чем я могу вам помочь?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 1578, 'total_tokens': 1598, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'gpt://b1gst3c7cskk2big5fqn/yandexgpt/rc', 'system_fingerprint': None, 'id': '08cee27c-2ddd-4cc6-8459-39634ba94554', 'finish_reason': 'stop', 'logprobs': None}, id='run-e291a3e7-892b-425f-871c-b63fba57dfc7-0', usage_metadata={'input_tokens': 1578, 'output_tokens': 20, 'total_tokens': 1598, 'input_token_details': {}, 'output_token_details': {}})]}
общежития и физкультуры нужна справка 086у. Достаточно будет отдать оригинал например а общежитие, в другое место копию? → {'messages': [AIMessage(content='Пожалуйста

CancelledError: 

In [ ]:
from ragas import EvaluationDataset
eval_ds = EvaluationDataset.from_list(dataset)


Оценка

In [ ]:
from ragas import evaluate
from ragas.llms import LangchainLLMWrapper
from ragas.metrics import LLMContextRecall, Faithfulness, ResponseRelevancy, AnswerCorrectness, AnswerCompleteness, ContextPrecision, ContextRecall
llm = ChatOpenAI(
        model="gpt://" + os.environ["FOLDER_ID"] + '/yandexgpt/rc',
        api_key=os.environ["API_KEY"],
        temperature=0.1,
        base_url="https://llm.api.cloud.yandex.net/v1"
    )
evaluator = LangchainLLMWrapper(llm)
metrics = [
    LLMContextRecall(),
    Faithfulness(),
    ResponseRelevancy(),
    AnswerCorrectness(),
    AnswerCompleteness(),
    ContextPrecision(),
    ContextRecall()
]
results = evaluate(
    dataset=eval_ds,
    metrics=metrics,
    llm=evaluator
)
print(results)
